In [1]:
import pandas as pd
import jieba
import matplotlib.pyplot as plt

#读取数据集
train_data=pd.read_csv('C:\\Users\\hjg\\OneDrive\\桌面\\对话意图识别\\train.csv',sep='\t',header=None)
test_data=pd.read_csv('C:\\Users\\hjg\\OneDrive\\桌面\\对话意图识别\\test.csv',sep='\t',header=None)
#分词
train_text=' '.join(list(train_data[0]))
train_words=jieba.cut(train_text)
#读取停用词
cn_stopwords = ' '.join(str(x) for x in pd.read_csv('https://mirror.coggle.club/stopwords/baidu_stopwords.txt', header=None)[0])
train_words=[x for x in train_words if x not in cn_stopwords]
train_words=[x for x in train_words if len(x)>1]
train_words=[x for x in train_words if not x.isdigit()]#剔除纯数字                    

# 统计每个单词出现的频次
from collections import Counter
train_words_freq=Counter(train_words)
# 过滤频次比较低的单词
train_words=[x for x in train_words if train_words_freq[x]>=5]

#定义一个字典
train_word_prior={}
#对于训练集的每个样本
for row in train_data.iloc[:].itertuples():
    # text与label分别对应这行的文本数据与标签数据
    text,label=row[1],row[2]
    
    #将每行的第一列文本数据分词
    words=jieba.cut(text)
    
    #保证每行的词出现在已经筛好的词汇表中
    words=[x for x in words if x in train_words]
    
    #如果words为0，代表这行数据是脏数据，不统计
    if len(words)==0:
        continue
    #对每个单词进行统计（计算）
    for word in words:
        #如果字典中未出现这个单词，定义total为0
        if word not in train_word_prior:
            train_word_prior[word]={"total":0}
        #如果字典中的单词未出现这个单词对应的标签，则打上标记，赋值为0
        if label not in train_word_prior[word]:
            train_word_prior[word][label]=0
    #计数
    train_word_prior[word][label]+=1
    train_word_prior[word]['total']+=1


# 转换成DataFrame表格形式并转置
train_word_prior=pd.DataFrame(train_word_prior).T
# 填充缺失值，处理单词在其他意图中没有出现的情况
train_word_prior.fillna(0,inplace=True)


#计算单词在每个类别出现的概率,得到每个单词所对应的意图
for category in train_data[1].unique():
    train_word_prior[category]/=train_word_prior['total']

# 每个意图所对应的单词
train_word_prior['category']=train_word_prior.columns[1:][train_word_prior.values[:,1:].argmax(1)]
train_word_prior.groupby('category').apply(lambda x:list(x.index))

df=train_word_prior.groupby('category').apply(lambda x:list(x.index))
intent_categories=df.index

#意图类别所对应的规则
import re
intent_patterns=[re.compile('|'.join(x)) for x in df.values]

test_pred=[]
for test in test_data[0]: 
    single_pred=''
    for category,pattern in zip(intent_categories,intent_patterns):
        if re.findall(pattern,test):
            single_pred=category
            break
    if single_pred=='':
        single_pred='Other'
    test_pred.append(single_pred)

pd.DataFrame(
    {
        "ID": range(1,len(test_data)+1),
        "Target": test_pred
    }
).to_csv('正则提交.csv',index=None)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hjg\AppData\Local\Temp\jieba.cache
Loading model cost 1.253 seconds.
Prefix dict has been built successfully.
C:\Users\hjg\AppData\Local\Temp\ipykernel_15016\3380694067.py:64: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_word_prior.groupby('category').apply(lambda x:list(x.index))
C:\Users\hjg\AppData\Local\Temp\ipykernel_15016\3380694067.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or ex